# Setup

In [ ]:
using CairoMakie

In [ ]:
using Arya

In [ ]:
import LilGuys as lguys

In [ ]:
out = lguys.Output("../models/tests/twobody/out/combined.hdf5")

In [ ]:
pos = lguys.extract_vector(out, :positions)

In [ ]:
acc = lguys.extract_vector(out, :accelerations)

In [ ]:
a = dropdims(sqrt.(sum(acc .^ 2, dims=1)), dims=1)[1, :]

In [ ]:
vec12 = pos[:, 1, :] .- pos[:, 2, :]

In [ ]:
r12 = dropdims(sqrt.(sum(vec12 .^ 2, dims=1)), dims=1)

In [ ]:
fig, ax = FigAxis(
    ylabel="rel acceleration error",
    xlabel="snapshot"
    )

scatter!(1 ./ r12 .^ 2 ./ a)

fig

In [ ]:
fig, ax = FigAxis(
    aspect=DataAspect(),
    xlabel="x",
    ylabel="y"
)

for i in 1:2
    scatter!(pos[1, i, :], pos[2, i, :])
end


fig

In [ ]:
fig, ax = FigAxis(
    aspect=DataAspect(),
    xlabel="time",
    ylabel="x"
)

for i in 1:2
    scatter!(out.times, pos[1, i, :])
end


fig

In [ ]:
ke = [sum(lguys.calc_E_spec_kin(s) .* s.masses) for s in out]
te = [lguys.calc_E_tot(s) for s in out]
pe = [sum(s.Φs .* s.masses)/2 for s in out];

In [ ]:
fig, ax = FigAxis(
    ylabel="Energy",
    xlabel="time"
)
scatter!(out.times, ke)
scatter!(out.times, pe)
scatter!(out.times, te)
scatter!(out.times, ke .+ pe)

fig

In [ ]:
fig, ax = FigAxis()
scatter!(out.times, te)
fig

Kepler's third law is 
$$
P^2 = \frac{4\pi^2}{\mu} a^3
$$ 
where $\mu = G(m+m)$. 

In [ ]:
out.times[sortperm(ke)]

In [ ]:
P = ans[end-1] - ans[end]

In [ ]:
maximum(pos[1, 1, :])

In [ ]:
minimum(pos[1, 1, :])

In [ ]:
# reduced mass
μ = out[1].masses[1]/2

# M1 frame
r12 = lguys.calc_r(pos[:, 1, :] .- pos[:, 2, :])
a = (maximum(r12) - minimum(r12)) / 2

In [ ]:
P, (out.times[2] - out.times[1])

In [ ]:
2π /μ^(1/2)  * a^(3/2)

# Threebody

In [ ]:
out = lguys.Output("../models/tests/threebody/out/combined.hdf5")

In [ ]:
pos = lguys.extract_vector(out, :positions)

In [ ]:
fig, ax = FigAxis()

for i in 1:3
    scatter!(pos[1, i, :], pos[2, i, :])
end

fig

In [ ]:
ke = [sum(lguys.calc_E_spec_kin(s) .* s.masses) for s in out]
te = [lguys.calc_E_tot(s) for s in out]
pe = [sum(s.Φs .* s.masses)/2 for s in out];

In [ ]:
fig, ax = FigAxis(
    ylabel="Energy",
    xlabel="time"
)
scatter!(out.times, ke)
scatter!(out.times, pe)
scatter!(out.times, te)
scatter!(out.times, ke .+ pe)

fig

# Potential tests

In [ ]:
import QuadGK: quadgk
import Roots: find_zero

In [ ]:
?lguys.calc_L_spec

In [ ]:
function calc_peri_apo_theory(pos, vel, Φ::Function)
    
    L0 = lguys.calc_r(lguys.calc_L_spec(pos[:, 1], vel[:, 1]))
    v0 = lguys.calc_r(vel)[1]
    rs = lguys.calc_r(pos)
    r0 = rs[1]
    
    E0 = 1/2 * v0^2 + Φ(r0)
    println("E0 ", E0)
    println("L0 ", L0)
    peri_obs = minimum(rs)
    apo_obs = maximum(rs)
    
    
    f(r) = r^-2 + 2*(Φ(r) - E0) / (L0^2)
    g(r) = 2 / sqrt(2 * (E0 - Φ(r)) - L0^2 / r^2)
    
    peri = find_zero(f, peri_obs)
    apo = find_zero(f, apo_obs)
    
    println("rel peri error ", peri_obs / peri)
    println("rel apo error ", apo_obs / apo)
    
    ϵ = 1e-5
    T_exp = quadgk(g, peri + ϵ, apo - ϵ)[1]
    
    return peri, apo, T_exp
end

Functions to calculate if the zeros are 

# Hernquist potential

In [ ]:
import TOML

## Special Cases

Input: a potential with total mass = 0.5 and scale radius 1

In [ ]:
readdir("/home/j/jfncc/dboyea/dwarfs/models/tests/hernquist/special_cases/")

In [ ]:
out = lguys.Output("/home/j/jfncc/dboyea/dwarfs/models/tests/hernquist/special_cases/out/combined.hdf5")

In [ ]:
pos = lguys.extract_vector(out, :positions)
vel = lguys.extract_vector(out, :velocities);

In [ ]:
fig, ax = FigAxis(aspect=DataAspect())

for i in 1:size(pos, 2)
    lines!(pos[1, i, :], pos[2, i, :])
end

fig

In [ ]:
function plot_E_tot(snap, idx=:)
end

In [ ]:
function plot_L(snap, idx=:)
end

## Random

In [ ]:
out = lguys.Output("/home/j/jfncc/dboyea/dwarfs/models/tests/hernquist/random/out/combined.hdf5")

In [ ]:
M = 0.9
r_s = 1.5

calc_Φ(r) = -lguys.G * M * 1/(r + r_s)

In [ ]:
idx = 102

In [ ]:
pos = lguys.extract_vector(out, :positions, idx)
vel = lguys.extract_vector(out, :velocities, idx);

In [ ]:
r = lguys.calc_r(pos);

In [ ]:
peri, apo, T = calc_peri_apo_theory(pos, vel, calc_Φ)

In [ ]:
fig, ax = FigAxis(xlabel="time / Gyr", ylabel = "r / kpc")

lines!(out.times * lguys.T0, r)
hlines!([peri, apo], color=COLORS[2], label="peri / apo")

t_first_peri = out.times[argmin(r[out.times .< T])]

N_periods = floor(Int, (out.times[end] - t_first_peri) / T)
vlines!((t_first_peri .+ collect(0:N_periods) * T) * lguys.T0, color=:black)
fig